In [199]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from scipy import stats
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from mlens.ensemble import SuperLearner
import pickle

In [2]:
train_data_values = pd.read_csv('./data/train_values.csv', index_col=0)
train_data_labels = pd.read_csv('./data/train_labels.csv', index_col=0)

train_data = pd.concat([train_data_values, train_data_labels], axis=1)

In [3]:
train_data.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


In [4]:
train_data.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [5]:
numeric_features = [
  'count_floors_pre_eq', 'age', 
  'area_percentage', 'height_percentage',
  'count_families'
]

categorical_features = [
  'land_surface_condition', 'foundation_type',
  'roof_type', 'ground_floor_type',
  'other_floor_type', 'position',
  'plan_configuration', 'legal_ownership_status'
]

other_features = list(np.setdiff1d(
  train_data.columns,
  categorical_features + ['damage_grade'],
  assume_unique=True
))

Let's see how imbalanced the data really is:

In [6]:
train_data[train_data['damage_grade'] == 1].shape

(25124, 39)

In [7]:
train_data[train_data['damage_grade'] == 2].shape

(148259, 39)

In [8]:
train_data[train_data['damage_grade'] == 3].shape

(87218, 39)

Let's see if we can improve the imbalance by removing outliers. 

In [9]:
train_data = train_data[(np.abs(stats.zscore(train_data[numeric_features])) < 3).all(axis=1)]

In [10]:
train_data[train_data['damage_grade'] == 1].shape

(22900, 39)

In [11]:
train_data[train_data['damage_grade'] == 2].shape

(142343, 39)

In [12]:
train_data[train_data['damage_grade'] == 3].shape

(84588, 39)

It has helped a little bit, but it also helps indirectly with balancing the data by other techniques later.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_data_values,
                                                    train_data_labels['damage_grade'],
                                                    test_size=0.25,
                                                    random_state=999)

In [14]:
X_train.head(10)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
985502,6,673,2937,2,5,13,6,t,i,x,...,0,0,0,0,0,0,0,0,0,0
140510,3,1387,2791,5,100,15,10,t,r,q,...,0,1,0,0,0,0,0,0,0,0
701507,11,765,3900,2,20,11,7,t,r,n,...,0,0,0,0,0,0,0,0,0,0
438410,17,1313,2490,3,0,8,6,t,r,q,...,0,0,0,0,0,0,0,0,0,0
800610,21,1219,2948,2,0,6,5,t,r,q,...,0,0,0,0,0,0,0,0,0,0
152270,22,763,4002,1,10,12,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0
511298,16,1399,4589,1,30,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,0
42065,10,337,296,2,5,5,5,t,r,q,...,0,0,0,0,0,0,0,0,0,0
669566,13,1154,11118,2,20,7,4,n,r,n,...,0,0,0,0,0,0,0,0,0,0


In [15]:
preprocessor = ColumnTransformer(
  transformers=[
      ('ohe', OneHotEncoder(drop="first"), categorical_features)
  ],
  remainder='passthrough'
)

In [16]:
X_train = pd.DataFrame(
  preprocessor.fit_transform(X_train),
  index=X_train.index,
  columns=(
    list(preprocessor.named_transformers_['ohe'].get_feature_names(categorical_features)) +
    other_features
  )
)

X_test = pd.DataFrame(preprocessor.transform(X_test),
                      index=X_test.index,
                      columns=X_train.columns)

In [17]:
X_train.head(10)

,land_surface_condition_o,land_surface_condition_t,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_q,roof_type_x,ground_floor_type_m,ground_floor_type_v,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
985502,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140510,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
701507,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438410,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
800610,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152270,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511298,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42065,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669566,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
sm = SMOTE(random_state=27)

In [19]:
X_train, y_train = sm.fit_sample(X_train, y_train)

In [20]:
parameters = { 
  'min_samples_split': [5, 10],
  'n_estimators': [100, 150]
}

In [21]:
rfc = GridSearchCV(
  RandomForestClassifier(), 
  parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] min_samples_split=5, n_estimators=100 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ min_samples_split=5, n_estimators=100, total= 1.1min
[CV] min_samples_split=5, n_estimators=100 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV] ............ min_samples_split=5, n_estimators=100, total= 1.1min
[CV] min_samples_split=5, n_estimators=100 ...........................
[CV] ............ min_samples_split=5, n_estimators=100, total= 1.0min
[CV] min_samples_split=5, n_estimators=100 ...........................
[CV] ............ min_samples_split=5, n_estimators=100, total= 1.0min
[CV] min_samples_split=5, n_estimators=100 ...........................
[CV] ............ min_samples_split=5, n_estimators=100, total= 1.1min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] ............ min_samples_split=5, n_estimators=150, total= 1.7min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] ............ min_samples_split=5, n_estimators=150, total= 1.6min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] ............ min_samples_split=5, n_estimators=150, total= 1.6min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 26.5min finished


In [22]:
dummy = DummyClassifier(strategy='uniform').fit(X_train, y_train)

In [23]:
rfc_pred = rfc.predict(X_test)

In [24]:
dummy_pred = dummy.predict(X_test)

In [25]:
f1_score(y_test, rfc_pred, average='micro')

0.7154456570121718

In [26]:
f1_score(y_test, dummy_pred, average='micro')

0.3321975103989194

In [27]:
rfc.best_params_

{'min_samples_split': 5, 'n_estimators': 150}

In [28]:
rfc.best_estimator_.feature_importances_

array([4.12697544e-03, 1.34810288e-02, 1.48282469e-02, 5.58917837e-02,
       7.12132740e-03, 1.57045446e-02, 1.81129986e-02, 2.44310584e-02,
       4.94440893e-04, 3.40336518e-02, 9.13055031e-03, 7.27331681e-04,
       2.86526569e-02, 5.89886443e-03, 1.01790457e-02, 9.01005288e-04,
       1.68189435e-02, 1.62162891e-02, 2.45230148e-04, 4.56663325e-03,
       1.14111400e-05, 3.00304655e-05, 5.99546937e-05, 1.35121410e-04,
       2.91398000e-03, 2.84988168e-04, 2.28845934e-03, 9.48629233e-04,
       4.89757230e-03, 1.38871781e-03, 1.19122582e-01, 8.80213876e-02,
       8.42721671e-02, 3.61010965e-02, 7.27864144e-02, 6.36890294e-02,
       4.75152927e-02, 9.86192223e-03, 5.67590125e-02, 7.08064322e-03,
       3.58430938e-03, 8.63231908e-03, 2.14022479e-02, 1.86385007e-02,
       9.54495662e-03, 8.64149719e-03, 5.23642474e-03, 3.25112912e-03,
       2.05913881e-02, 8.80682209e-03, 5.30510040e-03, 4.05387948e-03,
       1.06703636e-03, 1.66313150e-04, 6.80921112e-05, 2.04605176e-04,
      

In [29]:
X_test.columns

Index(['land_surface_condition_o', 'land_surface_condition_t',
       'foundation_type_i', 'foundation_type_r', 'foundation_type_u',
       'foundation_type_w', 'roof_type_q', 'roof_type_x',
       'ground_floor_type_m', 'ground_floor_type_v', 'ground_floor_type_x',
       'ground_floor_type_z', 'other_floor_type_q', 'other_floor_type_s',
       'other_floor_type_x', 'position_o', 'position_s', 'position_t',
       'plan_configuration_c', 'plan_configuration_d', 'plan_configuration_f',
       'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o',
       'plan_configuration_q', 'plan_configuration_s', 'plan_configuration_u',
       'legal_ownership_status_r', 'legal_ownership_status_v',
       'legal_ownership_status_w', 'geo_level_1_id', 'geo_level_2_id',
       'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage',
       'height_percentage', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
    

In [30]:
feature_importantance_series = pd.Series(rfc.best_estimator_.feature_importances_)

In [31]:
feature_importantance_series.index = X_test.columns

In [32]:
feature_importantance_series.sort_values(ascending=False)

geo_level_1_id                            0.119123
geo_level_2_id                            0.088021
geo_level_3_id                            0.084272
age                                       0.072786
area_percentage                           0.063689
has_superstructure_mud_mortar_stone       0.056759
foundation_type_r                         0.055892
height_percentage                         0.047515
count_floors_pre_eq                       0.036101
ground_floor_type_v                       0.034034
other_floor_type_q                        0.028653
roof_type_x                               0.024431
has_superstructure_cement_mortar_brick    0.021402
count_families                            0.020591
has_superstructure_timber                 0.018639
roof_type_q                               0.018113
position_s                                0.016819
position_t                                0.016216
foundation_type_w                         0.015705
foundation_type_i              

In [35]:
xgb_parameters = {
  'max_depth': [4],
  'colsample_bytree': [0.4, 0.8]
}

In [36]:
xgb_model = GridSearchCV(
  XGBClassifier(), 
  xgb_parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] colsample_bytree=0.4, max_depth=4 ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ colsample_bytree=0.4, max_depth=4, total= 3.0min
[CV] colsample_bytree=0.4, max_depth=4 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV] ................ colsample_bytree=0.4, max_depth=4, total= 2.8min
[CV] colsample_bytree=0.4, max_depth=4 ...............................
[CV] ................ colsample_bytree=0.4, max_depth=4, total= 2.7min
[CV] colsample_bytree=0.4, max_depth=4 ...............................
[CV] ................ colsample_bytree=0.4, max_depth=4, total= 2.9min
[CV] colsample_bytree=0.4, max_depth=4 ...............................
[CV] ................ colsample_bytree=0.4, max_depth=4, total= 3.0min
[CV] colsample_bytree=0.8, max_depth=4 ...............................
[CV] ................ colsample_bytree=0.8, max_depth=4, total= 4.5min
[CV] colsample_bytree=0.8, max_depth=4 ...............................
[CV] ................ colsample_bytree=0.8, max_depth=4, total= 4.4min
[CV] colsample_bytree=0.8, max_depth=4 ...............................
[CV] ................ colsample_bytree=0.8, max_depth=4, total= 4.3min
[CV] colsample_bytree=0.8, max_depth=4 ...............................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 36.3min finished


In [37]:
xgb_model.best_params_

{'colsample_bytree': 0.8, 'max_depth': 4}

In [38]:
xgb_model.best_estimator_.feature_importances_

array([0.00909472, 0.04117168, 0.01224456, 0.23470564, 0.02218183,
       0.01537039, 0.02123061, 0.00920259, 0.0008152 , 0.06663419,
       0.00279166, 0.0035749 , 0.04659707, 0.00586438, 0.00800305,
       0.00033032, 0.04717945, 0.01614195, 0.        , 0.00801948,
       0.        , 0.        , 0.        , 0.        , 0.00940653,
       0.        , 0.0132566 , 0.00035919, 0.00085206, 0.00058476,
       0.03031448, 0.00404248, 0.00151572, 0.03158781, 0.02925956,
       0.02077786, 0.01401757, 0.02070684, 0.06159075, 0.0150498 ,
       0.00653275, 0.02305559, 0.02726433, 0.0148351 , 0.01508106,
       0.00557674, 0.00642519, 0.00460602, 0.02135737, 0.04085347,
       0.00196232, 0.00283279, 0.00284439, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00229719],
      dtype=float32)

In [40]:
feature_importantance_series = pd.Series(xgb_model.best_estimator_.feature_importances_)
feature_importantance_series.index = X_test.columns
feature_importantance_series.sort_values(ascending=False)

foundation_type_r                         0.234706
ground_floor_type_v                       0.066634
has_superstructure_mud_mortar_stone       0.061591
position_s                                0.047179
other_floor_type_q                        0.046597
land_surface_condition_t                  0.041172
has_secondary_use                         0.040853
count_floors_pre_eq                       0.031588
geo_level_1_id                            0.030314
age                                       0.029260
has_superstructure_cement_mortar_brick    0.027264
has_superstructure_mud_mortar_brick       0.023056
foundation_type_u                         0.022182
count_families                            0.021357
roof_type_q                               0.021231
area_percentage                           0.020778
has_superstructure_adobe_mud              0.020707
position_t                                0.016142
foundation_type_w                         0.015370
has_superstructure_bamboo      

In [43]:
xgb_pred = xgb_model.predict(X_test.as_matrix())

/Users/jackyho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [44]:
f1_score(y_test, xgb_pred, average='micro')

0.6717471719543829

In [46]:
light_parameters = {
  'num_leaves': [6, 12],
  'min_data_in_leaf': [40, 80]
}

In [47]:
light_model = GridSearchCV(
  LGBMClassifier(), 
  light_parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] min_data_in_leaf=40, num_leaves=6 ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ min_data_in_leaf=40, num_leaves=6, total=  11.7s
[CV] min_data_in_leaf=40, num_leaves=6 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.7s remaining:    0.0s


[CV] ................ min_data_in_leaf=40, num_leaves=6, total=  11.8s
[CV] min_data_in_leaf=40, num_leaves=6 ...............................
[CV] ................ min_data_in_leaf=40, num_leaves=6, total=  11.6s
[CV] min_data_in_leaf=40, num_leaves=6 ...............................
[CV] ................ min_data_in_leaf=40, num_leaves=6, total=  13.9s
[CV] min_data_in_leaf=40, num_leaves=6 ...............................
[CV] ................ min_data_in_leaf=40, num_leaves=6, total=  12.2s
[CV] min_data_in_leaf=40, num_leaves=12 ..............................
[CV] ............... min_data_in_leaf=40, num_leaves=12, total=  16.7s
[CV] min_data_in_leaf=40, num_leaves=12 ..............................
[CV] ............... min_data_in_leaf=40, num_leaves=12, total=  16.9s
[CV] min_data_in_leaf=40, num_leaves=12 ..............................
[CV] ............... min_data_in_leaf=40, num_leaves=12, total=  15.6s
[CV] min_data_in_leaf=40, num_leaves=12 ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  4.7min finished


In [48]:
feature_importantance_series = pd.Series(light_model.best_estimator_.feature_importances_)
feature_importantance_series.index = X_test.columns
feature_importantance_series.sort_values(ascending=False)

geo_level_1_id                            901
age                                       300
geo_level_2_id                            220
area_percentage                           203
count_families                            137
count_floors_pre_eq                       102
has_superstructure_cement_mortar_brick     89
height_percentage                          88
has_superstructure_mud_mortar_stone        84
roof_type_q                                81
foundation_type_r                          77
other_floor_type_q                         74
position_s                                 67
has_secondary_use                          66
position_t                                 60
ground_floor_type_v                        54
has_superstructure_stone_flag              52
has_superstructure_timber                  51
land_surface_condition_t                   51
foundation_type_i                          44
roof_type_x                                44
has_superstructure_mud_mortar_bric

In [49]:
light_model.best_params_

{'min_data_in_leaf': 80, 'num_leaves': 12}

In [50]:
light_pred = light_model.predict(X_test)

In [51]:
f1_score(y_test, light_pred, average='micro')

0.6740648646989302

Gather all the models for stacking

In [52]:
rf_model = RandomForestClassifier(min_samples_split = 5, n_estimators = 150).fit(X_train, y_train)
xgb_model = XGBClassifier(colsample_bytree = 0.8, max_depth = 4).fit(X_train, y_train)
light_model = LGBMClassifier(min_data_in_leaf = 80, num_leaves = 12).fit(X_train, y_train)

In [64]:
stacking_train_df = pd.DataFrame({
 'rf_pred': rf_model.predict(X_train),
  'xgb_pred': xgb_model.predict(X_train),
  'light_pred': light_model.predict(X_train)
})

In [65]:
stacking_test_df = pd.DataFrame({
 'rf_pred': rf_model.predict(X_test),
  'xgb_pred': xgb_model.predict(X_test.as_matrix()),
  'light_pred': light_model.predict(X_test)
})

/Users/jackyho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
stacking_train_df

,rf_pred,xgb_pred,light_pred
0,1,1,1
1,2,2,2
2,3,2,3
3,3,3,3
4,3,3,3
...,...,...,...
333886,3,3,3
333887,3,3,3
333888,3,3,3
333889,3,3,3


In [90]:
cat_parameters = {
  'depth': [5, 10],
  'learning_rate': [0.1, 0.3], 
  'l2_leaf_reg': [10, 100],
  'one_hot_max_size': [5]
}

In [91]:
cat_model = GridSearchCV(
  CatBoostClassifier(), 
  cat_parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(stacking_train_df, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] depth=5, l2_leaf_reg=10, learning_rate=0.1, one_hot_max_size=5 ..


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.9206716	total: 113ms	remaining: 1m 53s
1:	learn: 0.7883997	total: 152ms	remaining: 1m 15s
2:	learn: 0.6851789	total: 194ms	remaining: 1m 4s
3:	learn: 0.6020655	total: 236ms	remaining: 58.7s
4:	learn: 0.5337080	total: 271ms	remaining: 54s
5:	learn: 0.4766734	total: 324ms	remaining: 53.7s
6:	learn: 0.4284259	total: 374ms	remaining: 53.1s
7:	learn: 0.3872946	total: 421ms	remaining: 52.2s
8:	learn: 0.3519611	total: 462ms	remaining: 50.9s
9:	learn: 0.3214290	total: 501ms	remaining: 49.6s
10:	learn: 0.2949236	total: 544ms	remaining: 48.9s
11:	learn: 0.2718342	total: 588ms	remaining: 48.4s
12:	learn: 0.2516536	total: 633ms	remaining: 48.1s
13:	learn: 0.2339613	total: 677ms	remaining: 47.6s
14:	learn: 0.2184225	total: 712ms	remaining: 46.7s
15:	learn: 0.2047424	total: 753ms	remaining: 46.3s
16:	learn: 0.1927017	total: 804ms	remaining: 46.5s
17:	learn: 0.1820771	total: 841ms	remaining: 45.9s
18:	learn: 0.1726621	total: 885ms	remaining: 45.7s
19:	learn: 0.1643712	total: 920ms	remaini

161:	learn: 0.0991821	total: 7.78s	remaining: 40.3s
162:	learn: 0.0991816	total: 7.84s	remaining: 40.3s
163:	learn: 0.0991813	total: 7.89s	remaining: 40.2s
164:	learn: 0.0991810	total: 7.94s	remaining: 40.2s
165:	learn: 0.0991806	total: 7.98s	remaining: 40.1s
166:	learn: 0.0991802	total: 8.03s	remaining: 40.1s
167:	learn: 0.0991799	total: 8.1s	remaining: 40.1s
168:	learn: 0.0991796	total: 8.16s	remaining: 40.1s
169:	learn: 0.0991793	total: 8.22s	remaining: 40.1s
170:	learn: 0.0991790	total: 8.28s	remaining: 40.1s
171:	learn: 0.0991786	total: 8.36s	remaining: 40.2s
172:	learn: 0.0991783	total: 8.43s	remaining: 40.3s
173:	learn: 0.0991780	total: 8.5s	remaining: 40.3s
174:	learn: 0.0991777	total: 8.58s	remaining: 40.5s
175:	learn: 0.0991774	total: 8.65s	remaining: 40.5s
176:	learn: 0.0991770	total: 8.71s	remaining: 40.5s
177:	learn: 0.0991767	total: 8.76s	remaining: 40.5s
178:	learn: 0.0991764	total: 8.82s	remaining: 40.4s
179:	learn: 0.0991762	total: 8.87s	remaining: 40.4s
180:	learn: 0.

322:	learn: 0.0991594	total: 16.8s	remaining: 35.2s
323:	learn: 0.0991593	total: 16.8s	remaining: 35.1s
324:	learn: 0.0991593	total: 16.9s	remaining: 35.1s
325:	learn: 0.0991592	total: 16.9s	remaining: 35s
326:	learn: 0.0991592	total: 17s	remaining: 35s
327:	learn: 0.0991591	total: 17.1s	remaining: 34.9s
328:	learn: 0.0991591	total: 17.1s	remaining: 34.9s
329:	learn: 0.0991590	total: 17.2s	remaining: 34.9s
330:	learn: 0.0991590	total: 17.3s	remaining: 34.9s
331:	learn: 0.0991589	total: 17.3s	remaining: 34.8s
332:	learn: 0.0991589	total: 17.4s	remaining: 34.8s
333:	learn: 0.0991588	total: 17.4s	remaining: 34.7s
334:	learn: 0.0991588	total: 17.5s	remaining: 34.7s
335:	learn: 0.0991588	total: 17.5s	remaining: 34.7s
336:	learn: 0.0991587	total: 17.6s	remaining: 34.6s
337:	learn: 0.0991587	total: 17.6s	remaining: 34.5s
338:	learn: 0.0991586	total: 17.7s	remaining: 34.5s
339:	learn: 0.0991586	total: 17.8s	remaining: 34.5s
340:	learn: 0.0991585	total: 17.8s	remaining: 34.4s
341:	learn: 0.0991

...

323:	learn: 0.1382042	total: 17.9s	remaining: 37.4s
324:	learn: 0.1382041	total: 18s	remaining: 37.3s
325:	learn: 0.1382039	total: 18s	remaining: 37.3s
326:	learn: 0.1382038	total: 18.1s	remaining: 37.2s
327:	learn: 0.1382037	total: 18.1s	remaining: 37.2s
328:	learn: 0.1382035	total: 18.2s	remaining: 37.1s
329:	learn: 0.1382034	total: 18.3s	remaining: 37.1s
330:	learn: 0.1382032	total: 18.3s	remaining: 37s
331:	learn: 0.1382031	total: 18.4s	remaining: 37s
332:	learn: 0.1382030	total: 18.4s	remaining: 36.9s
333:	learn: 0.1382029	total: 18.5s	remaining: 36.8s
334:	learn: 0.1382027	total: 18.5s	remaining: 36.8s
335:	learn: 0.1382026	total: 18.6s	remaining: 36.7s
336:	learn: 0.1382025	total: 18.6s	remaining: 36.7s
337:	learn: 0.1382023	total: 18.7s	remaining: 36.6s
338:	learn: 0.1382022	total: 18.7s	remaining: 36.6s
339:	learn: 0.1382020	total: 18.8s	remaining: 36.5s
340:	learn: 0.1382019	total: 18.9s	remaining: 36.5s
341:	learn: 0.1382018	total: 18.9s	remaining: 36.4s
342:	learn: 0.138201

483:	learn: 0.1381895	total: 26.9s	remaining: 28.7s
484:	learn: 0.1381894	total: 27s	remaining: 28.7s
485:	learn: 0.1381894	total: 27s	remaining: 28.6s
486:	learn: 0.1381893	total: 27.1s	remaining: 28.5s
487:	learn: 0.1381892	total: 27.1s	remaining: 28.5s
488:	learn: 0.1381892	total: 27.2s	remaining: 28.4s
489:	learn: 0.1381891	total: 27.3s	remaining: 28.4s
490:	learn: 0.1381890	total: 27.3s	remaining: 28.3s
491:	learn: 0.1381890	total: 27.4s	remaining: 28.3s
492:	learn: 0.1381889	total: 27.4s	remaining: 28.2s
493:	learn: 0.1381889	total: 27.5s	remaining: 28.2s
494:	learn: 0.1381888	total: 27.5s	remaining: 28.1s
495:	learn: 0.1381888	total: 27.6s	remaining: 28s
496:	learn: 0.1381887	total: 27.7s	remaining: 28s
497:	learn: 0.1381887	total: 27.7s	remaining: 27.9s
498:	learn: 0.1381886	total: 27.8s	remaining: 27.9s
499:	learn: 0.1381885	total: 27.8s	remaining: 27.8s
500:	learn: 0.1381885	total: 27.9s	remaining: 27.8s
501:	learn: 0.1381884	total: 27.9s	remaining: 27.7s
502:	learn: 0.138188

643:	learn: 0.1381822	total: 36s	remaining: 19.9s
644:	learn: 0.1381822	total: 36s	remaining: 19.8s
645:	learn: 0.1381821	total: 36.1s	remaining: 19.8s
646:	learn: 0.1381821	total: 36.1s	remaining: 19.7s
647:	learn: 0.1381821	total: 36.2s	remaining: 19.7s
648:	learn: 0.1381820	total: 36.3s	remaining: 19.6s
649:	learn: 0.1381820	total: 36.3s	remaining: 19.6s
650:	learn: 0.1381819	total: 36.4s	remaining: 19.5s
651:	learn: 0.1381819	total: 36.5s	remaining: 19.5s
652:	learn: 0.1381819	total: 36.5s	remaining: 19.4s
653:	learn: 0.1381818	total: 36.6s	remaining: 19.4s
654:	learn: 0.1381818	total: 36.7s	remaining: 19.3s
655:	learn: 0.1381818	total: 36.7s	remaining: 19.3s
656:	learn: 0.1381818	total: 36.8s	remaining: 19.2s
657:	learn: 0.1381817	total: 36.9s	remaining: 19.2s
658:	learn: 0.1381817	total: 36.9s	remaining: 19.1s
659:	learn: 0.1381817	total: 37s	remaining: 19.1s
660:	learn: 0.1381816	total: 37s	remaining: 19s
661:	learn: 0.1381816	total: 37.1s	remaining: 18.9s
662:	learn: 0.1381816	

802:	learn: 0.1381779	total: 45.7s	remaining: 11.2s
803:	learn: 0.1381778	total: 45.8s	remaining: 11.2s
804:	learn: 0.1381778	total: 45.8s	remaining: 11.1s
805:	learn: 0.1381778	total: 45.9s	remaining: 11s
806:	learn: 0.1381778	total: 45.9s	remaining: 11s
807:	learn: 0.1381778	total: 46s	remaining: 10.9s
808:	learn: 0.1381777	total: 46.1s	remaining: 10.9s
809:	learn: 0.1381777	total: 46.1s	remaining: 10.8s
810:	learn: 0.1381777	total: 46.2s	remaining: 10.8s
811:	learn: 0.1381777	total: 46.2s	remaining: 10.7s
812:	learn: 0.1381777	total: 46.3s	remaining: 10.6s
813:	learn: 0.1381776	total: 46.3s	remaining: 10.6s
814:	learn: 0.1381776	total: 46.4s	remaining: 10.5s
815:	learn: 0.1381776	total: 46.5s	remaining: 10.5s
816:	learn: 0.1381776	total: 46.5s	remaining: 10.4s
817:	learn: 0.1381775	total: 46.6s	remaining: 10.4s
818:	learn: 0.1381775	total: 46.6s	remaining: 10.3s
819:	learn: 0.1381775	total: 46.7s	remaining: 10.2s
820:	learn: 0.1381775	total: 46.7s	remaining: 10.2s
821:	learn: 0.1381

961:	learn: 0.1381749	total: 54.7s	remaining: 2.16s
962:	learn: 0.1381749	total: 54.8s	remaining: 2.1s
963:	learn: 0.1381749	total: 54.9s	remaining: 2.05s
964:	learn: 0.1381749	total: 54.9s	remaining: 1.99s
965:	learn: 0.1381749	total: 55s	remaining: 1.93s
966:	learn: 0.1381748	total: 55s	remaining: 1.88s
967:	learn: 0.1381748	total: 55.1s	remaining: 1.82s
968:	learn: 0.1381748	total: 55.1s	remaining: 1.76s
969:	learn: 0.1381748	total: 55.2s	remaining: 1.71s
970:	learn: 0.1381748	total: 55.3s	remaining: 1.65s
971:	learn: 0.1381748	total: 55.3s	remaining: 1.59s
972:	learn: 0.1381747	total: 55.4s	remaining: 1.54s
973:	learn: 0.1381747	total: 55.4s	remaining: 1.48s
974:	learn: 0.1381747	total: 55.5s	remaining: 1.42s
975:	learn: 0.1381747	total: 55.5s	remaining: 1.36s
976:	learn: 0.1381747	total: 55.6s	remaining: 1.31s
977:	learn: 0.1381747	total: 55.6s	remaining: 1.25s
978:	learn: 0.1381747	total: 55.7s	remaining: 1.19s
979:	learn: 0.1381746	total: 55.8s	remaining: 1.14s
980:	learn: 0.138

In [98]:
cat_model.best_params_

{'depth': 5, 'l2_leaf_reg': 100, 'learning_rate': 0.1, 'one_hot_max_size': 5}

In [99]:
cat_pred = cat_model.predict(stacking_test_df)

In [100]:
f1_score(y_test, cat_pred, average='micro')

0.7163665945265614

In [227]:
from mlens.metrics import make_scorer
accuracy_scorer = make_scorer(f1_score, average='micro')

In [228]:
ensemble = SuperLearner(scorer=accuracy_scorer, random_state=123, verbose=2)

In [229]:
ensemble.add([
  RandomForestClassifier(min_samples_split=5, n_estimators=150, random_state=123), 
  XGBClassifier(colsample_bytree=0.8, max_depth=4, seed=123),
  LGBMClassifier(min_data_in_leaf = 80, num_leaves = 12, random_state=123)
])

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=3582, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...=micro))],
   n_jobs=-1, name='group-20', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=123, sample_size=20,
       scorer=make_scorer(f1_score, average=micro), shuffle=False,
       verbose=2)

In [230]:
ensemble.add_meta(CatBoostClassifier(
  random_seed=123,
  depth=5, 
  l2_leaf_reg=100, 
  learning_rate=0.1, 
  one_hot_max_size=5
))

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=3582, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...=micro))],
   n_jobs=-1, name='group-21', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=123, sample_size=20,
       scorer=make_scorer(f1_score, average=micro), shuffle=False,
       verbose=2)

In [231]:
sub_preprocessor = ColumnTransformer(
  transformers=[('ohe', OneHotEncoder(drop="first"), categorical_features)],
  remainder='passthrough'
)

In [232]:
X = sub_preprocessor.fit_transform(train_data_values)

In [233]:
sm = SMOTE(random_state=27)

X_train, y_train = sm.fit_sample(X, train_data_labels['damage_grade'])

In [234]:
ensemble.fit(X_train, y_train)


Fitting 2 layers
Processing layer-1             

/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [lgbmclassifier.0.2] Could not score lgbmclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [lgbmclassifier.0.1] Could not score lgbmclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [randomforestclassifier.0.2] Could not score randomforestclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [randomforestclassifier.0.1]

done | 00:09:35
Processing layer-2             0:	learn: 1.0285146	total: 91.9ms	remaining: 1m 31s
1:	learn: 0.9747525	total: 163ms	remaining: 1m 21s
2:	learn: 0.9311384	total: 238ms	remaining: 1m 18s
3:	learn: 0.8952714	total: 321ms	remaining: 1m 19s
4:	learn: 0.8658990	total: 411ms	remaining: 1m 21s
5:	learn: 0.8411133	total: 492ms	remaining: 1m 21s
6:	learn: 0.8204324	total: 588ms	remaining: 1m 23s
7:	learn: 0.8028998	total: 667ms	remaining: 1m 22s
8:	learn: 0.7877418	total: 746ms	remaining: 1m 22s
9:	learn: 0.7748996	total: 837ms	remaining: 1m 22s
10:	learn: 0.7638734	total: 920ms	remaining: 1m 22s
11:	learn: 0.7543985	total: 998ms	remaining: 1m 22s
12:	learn: 0.7460562	total: 1.1s	remaining: 1m 23s
13:	learn: 0.7389177	total: 1.2s	remaining: 1m 24s
14:	learn: 0.7334612	total: 1.27s	remaining: 1m 23s
15:	learn: 0.7279191	total: 1.36s	remaining: 1m 23s
16:	learn: 0.7231127	total: 1.45s	remaining: 1m 23s
17:	learn: 0.7190659	total: 1.52s	remaining: 1m 23s
18:	learn: 0.7154789	total: 

157:	learn: 0.6913942	total: 12.6s	remaining: 1m 7s
158:	learn: 0.6913941	total: 12.7s	remaining: 1m 7s
159:	learn: 0.6913940	total: 12.8s	remaining: 1m 7s
160:	learn: 0.6913939	total: 12.9s	remaining: 1m 7s
161:	learn: 0.6913938	total: 12.9s	remaining: 1m 6s
162:	learn: 0.6913937	total: 13s	remaining: 1m 6s
163:	learn: 0.6913936	total: 13.1s	remaining: 1m 6s
164:	learn: 0.6913935	total: 13.2s	remaining: 1m 6s
165:	learn: 0.6913934	total: 13.3s	remaining: 1m 6s
166:	learn: 0.6913933	total: 13.4s	remaining: 1m 6s
167:	learn: 0.6913932	total: 13.4s	remaining: 1m 6s
168:	learn: 0.6913930	total: 13.5s	remaining: 1m 6s
169:	learn: 0.6913929	total: 13.6s	remaining: 1m 6s
170:	learn: 0.6913929	total: 13.7s	remaining: 1m 6s
171:	learn: 0.6913927	total: 13.8s	remaining: 1m 6s
172:	learn: 0.6913926	total: 13.8s	remaining: 1m 6s
173:	learn: 0.6913925	total: 13.9s	remaining: 1m 6s
174:	learn: 0.6913924	total: 14s	remaining: 1m 5s
175:	learn: 0.6913923	total: 14.1s	remaining: 1m 5s
176:	learn: 0.69

319:	learn: 0.6913867	total: 25.8s	remaining: 54.9s
320:	learn: 0.6913867	total: 25.9s	remaining: 54.8s
321:	learn: 0.6913867	total: 26s	remaining: 54.7s
322:	learn: 0.6913867	total: 26.1s	remaining: 54.6s
323:	learn: 0.6913867	total: 26.1s	remaining: 54.5s
324:	learn: 0.6913867	total: 26.2s	remaining: 54.5s
325:	learn: 0.6913866	total: 26.3s	remaining: 54.4s
326:	learn: 0.6913866	total: 26.4s	remaining: 54.3s
327:	learn: 0.6913866	total: 26.5s	remaining: 54.2s
328:	learn: 0.6913866	total: 26.6s	remaining: 54.2s
329:	learn: 0.6913866	total: 26.6s	remaining: 54.1s
330:	learn: 0.6913866	total: 26.7s	remaining: 54s
331:	learn: 0.6913866	total: 26.8s	remaining: 53.9s
332:	learn: 0.6913866	total: 26.9s	remaining: 53.8s
333:	learn: 0.6913865	total: 27s	remaining: 53.8s
334:	learn: 0.6913865	total: 27.1s	remaining: 53.7s
335:	learn: 0.6913865	total: 27.1s	remaining: 53.6s
336:	learn: 0.6913865	total: 27.2s	remaining: 53.5s
337:	learn: 0.6913865	total: 27.3s	remaining: 53.5s
338:	learn: 0.6913

...

801:	learn: 0.6913849	total: 1m 5s	remaining: 16.1s
802:	learn: 0.6913849	total: 1m 5s	remaining: 16s
803:	learn: 0.6913849	total: 1m 5s	remaining: 15.9s
804:	learn: 0.6913849	total: 1m 5s	remaining: 15.9s
805:	learn: 0.6913849	total: 1m 5s	remaining: 15.8s
806:	learn: 0.6913849	total: 1m 5s	remaining: 15.7s
807:	learn: 0.6913849	total: 1m 5s	remaining: 15.6s
808:	learn: 0.6913849	total: 1m 5s	remaining: 15.5s
809:	learn: 0.6913849	total: 1m 5s	remaining: 15.4s
810:	learn: 0.6913848	total: 1m 5s	remaining: 15.4s
811:	learn: 0.6913848	total: 1m 5s	remaining: 15.3s
812:	learn: 0.6913848	total: 1m 6s	remaining: 15.2s
813:	learn: 0.6913848	total: 1m 6s	remaining: 15.1s
814:	learn: 0.6913848	total: 1m 6s	remaining: 15s
815:	learn: 0.6913848	total: 1m 6s	remaining: 14.9s
816:	learn: 0.6913848	total: 1m 6s	remaining: 14.9s
817:	learn: 0.6913848	total: 1m 6s	remaining: 14.8s
818:	learn: 0.6913848	total: 1m 6s	remaining: 14.7s
819:	learn: 0.6913848	total: 1m 6s	remaining: 14.6s
820:	learn: 0.69

960:	learn: 0.6913848	total: 1m 18s	remaining: 3.17s
961:	learn: 0.6913848	total: 1m 18s	remaining: 3.08s
962:	learn: 0.6913848	total: 1m 18s	remaining: 3s
963:	learn: 0.6913848	total: 1m 18s	remaining: 2.92s
964:	learn: 0.6913848	total: 1m 18s	remaining: 2.84s
965:	learn: 0.6913848	total: 1m 18s	remaining: 2.76s
966:	learn: 0.6913848	total: 1m 18s	remaining: 2.68s
967:	learn: 0.6913848	total: 1m 18s	remaining: 2.6s
968:	learn: 0.6913848	total: 1m 18s	remaining: 2.52s
969:	learn: 0.6913848	total: 1m 18s	remaining: 2.44s
970:	learn: 0.6913848	total: 1m 18s	remaining: 2.35s
971:	learn: 0.6913848	total: 1m 18s	remaining: 2.27s
972:	learn: 0.6913848	total: 1m 19s	remaining: 2.19s
973:	learn: 0.6913848	total: 1m 19s	remaining: 2.11s
974:	learn: 0.6913848	total: 1m 19s	remaining: 2.03s
975:	learn: 0.6913848	total: 1m 19s	remaining: 1.95s
976:	learn: 0.6913848	total: 1m 19s	remaining: 1.87s
977:	learn: 0.6913848	total: 1m 19s	remaining: 1.79s
978:	learn: 0.6913848	total: 1m 19s	remaining: 1.7

/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [catboostclassifier.0.0] Could not score catboostclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)


done | 00:01:23
Fit complete                        | 00:10:59


SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=3582, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...=micro))],
   n_jobs=-1, name='group-21', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=123, sample_size=20,
       scorer=make_scorer(f1_score, average=micro), shuffle=False,
       verbose=2)

In [235]:
test_data_values = pd.read_csv('./data/test_values.csv', index_col=0)

In [236]:
X_test = sub_preprocessor.transform(test_data_values)

In [237]:
preds = ensemble.predict(X_test)


Predicting 2 layers


/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:313: ParameterChangeWarning: Parameter value ('multi:softprob') has changed since model was fitted ('binary:logistic').
  (lpar, rpar), ParameterChangeWarning)


Processing layer-1             done | 00:00:08
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:08


In [238]:
sub = pd.DataFrame({
  'building_id': test_data_values.index,
  'damage_grade': map(int, preds)
})

In [240]:
sub.to_csv('submission', index=False)